In [1]:
using Pkg
using LinearAlgebra
# using Plots
using Distributions
using Random
using StateSpaceDynamics
using StatsBase

const SSD = StateSpaceDynamics

StateSpaceDynamics

# Gaussian HMM

In [2]:
"""
Create an underlying GaussianHMM to generate data
"""

# Create Guassian Emission Models
output_dim = 2
μ = [0.0, 0.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(output_dim, μ, Σ)

μ = [2.0, 1.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(output_dim, μ, Σ)

# Create GaussianHMM
true_model = SSD.GaussianHMM(K=2, output_dim=2)
true_model.B[1] = emission_1
true_model.B[2] = emission_2
true_model.A = [0.9 0.1; 0.8 0.2]

# Sample from the model
n=10000
true_labels, data = SSD.sample(true_model, n=n)

# Fit a gaussian hmm to the data
test_model = SSD.GaussianHMM(K=2, output_dim=2)
test_model.A = [0.8 0.2; 0.05 0.95]
ll = SSD.fit!(test_model, data)

print(isapprox(test_model.B[1].μ, true_model.B[1].μ, atol=0.1) || isapprox(test_model.B[1].μ, true_model.B[2].μ, atol=0.1))
print(isapprox(test_model.B[2].μ, true_model.B[2].μ, atol=0.1) || isapprox(test_model.B[2].μ, true_model.B[1].μ, atol=0.1))

Running EM algorithm... 100%|██████████████████████████████████████████████████| Time: 0:00:22 ( 0.22  s/it)


truetrue

In [ ]:
# lls = SSD.emission_loglikelihoods(test_model, permutedims(data))
# α = SSD.forward(test_model, lls)
# β = SSD.backward(test_model, lls)
# γ = SSD.calculate_γ(test_model, α, β)
# ξ = SSD.calculate_ξ(test_model, α, β, lls)
γ, ξ, α, β = SSD.estep(test_model, (data,))


([-0.5372440468090645 -0.7168921322117967 … -1.60943791243335 -1.609437912436988; -0.8779273437648953 -0.6699529989091388 … -0.22314355131311459 -0.22314355131311459], [-0.760387598122179 -2.1466819592424145; -3.8736596173184807 -0.9292206381542201;;; -0.9400356835285493 -2.3263300446451467; -3.665685272466362 -0.7212462932948256;;; -1.0997316475040861 -2.4860260086206836; -3.5339299986953847 -0.5894910195238481;;; … ;;; -1.8325814637501026 -3.2188758248667; -3.2188758248667 -0.2744368457024393;;; -1.8325814637537405 -3.218875824870338; -3.2188758248667 -0.2744368457024393;;; -1.8325814637501026 -3.2188758248667; -3.2188758248667 -0.2744368457024393], [-5.267286702587403 -7.438494613951806 … -22139.192720225157 -22141.049279804505; -5.607969999546872 -7.391555480649791 … -22137.806425864037 -22139.66298544338], [-22134.709799236916 -22132.71823941095 … -1.856559579344148 0.0; -22134.709799236913 -22132.71823941095 … -1.8565595793446803 0.0])

In [4]:
plot(ll)

UndefVarError: UndefVarError: `plot` not defined

# Switching Gaussian Regression #

In [5]:
# Create Emission Models
emission_1 = GaussianRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([3, 2, 2, 3], :, 1))
emission_2 = GaussianRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([-4, -2, 3, 2], :, 1))

# Create Switching Regression Model
true_model = SwitchingGaussianRegression(K=2, input_dim=3, output_dim=1, include_intercept=true)

# Plug in the emission models
true_model.B[1] = emission_1
true_model.B[2] = emission_2

# Sample from the model
n = 20000
Φ = randn(n, 3)
true_labels, data = SSD.sample(true_model, Φ, n=n)

# Try to fit a new model to the data
test_model = SSD.SwitchingGaussianRegression(K=2, input_dim=3, output_dim=1, include_intercept=true)
ll = SSD.fit!(test_model, data, Φ)

# Test output -> not quite right yet
print(isapprox(test_model.B[1].β, true_model.B[1].β, atol=0.1) || isapprox(test_model.B[1].β, true_model.B[2].β, atol=0.1))
print(isapprox(test_model.B[2].β, true_model.B[2].β, atol=0.1) || isapprox(test_model.B[2].β, true_model.B[1].β, atol=0.1))

DimensionMismatch: DimensionMismatch: matrix A has dimensions (1,20001), matrix B has dimensions (4,1)

In [6]:
println(true_model.A)
println(test_model.A)

[0.9386837913023849 0.06131620869761511; 0.9058390587776387 0.09416094122236134]
[0.8 0.2; 0.05 0.95]


In [7]:
plot(ll)

UndefVarError: UndefVarError: `plot` not defined

# Switching Bernoulli Regression #

In [8]:
"""
Create Bernoulli Regression Model
"""
# Make Emission Models
emission_1 = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([3, 1, 2], :, 1))
emission_2 = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-3, -2, 0.1], :, 1))

# Create Switching Bernoulli Regression and add the emissions
true_model = SSD.SwitchingBernoulliRegression(K=2, input_dim=2)
true_model.A = [0.9 0.1; 0.2 0.8]
true_model.B[1] = emission_1
true_model.B[2] = emission_2

# Sample from the model
n=40000
Φ = randn(n, 2)
true_labels, data = SSD.sample(true_model, Φ, n=n)

# Fit a new Bernoulli Regression Model to the data
test_model = SSD.SwitchingBernoulliRegression(K=2, input_dim=2, λ=1.0)
test_model.A = [0.75 0.25; 0.1 0.9]
test_model.B[1] = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([2.5, 0.25, 1.0], :, 1))
test_model.B[2] = SSD.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-2.0, -3.0, -1.0], :, 1))
ll = SSD.fit!(test_model, data, Φ, max_iters=200)

# # Test it works alright
print(isapprox(test_model.B[1].β, true_model.B[1].β, atol=0.2) || isapprox(test_model.B[1].β, true_model.B[2].β, atol=0.2))
print(isapprox(test_model.B[2].β, true_model.B[2].β, atol=0.2) || isapprox(test_model.B[2].β, true_model.B[1].β, atol=0.2))

DimensionMismatch: DimensionMismatch: matrix A has dimensions (1,40000), matrix B has dimensions (3,1)

In [9]:
 # Make Emission Models
 emission_1 = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([3, 1, 2], :, 1))
 emission_2 = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-3, -2, 0.1], :, 1))

 # Create Switching Bernoulli Regression and add the emissions
 true_model = StateSpaceDynamics.SwitchingBernoulliRegression(K=2, input_dim=2)
 true_model.A = [0.9 0.1; 0.2 0.8]
 true_model.B[1] = emission_1
 true_model.B[2] = emission_2

 # Sample from the model
 n=20000
 Φ = randn(n, 2)
 true_labels, data = StateSpaceDynamics.sample(true_model, Φ, n=n)

 # Fit a new Bernoulli Regression Model to the data
 test_model = StateSpaceDynamics.SwitchingBernoulliRegression(K=2, input_dim=2, λ=1.0)
 test_model.A = [0.75 0.25; 0.1 0.9]
 test_model.B[1] = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([2.5, 0.25, 1.0], :, 1))
 test_model.B[2] = StateSpaceDynamics.BernoulliRegressionEmission(input_dim=2, output_dim=1, include_intercept=true, β = reshape([-2.0, -3.0, -1.0], :, 1))
 ll = StateSpaceDynamics.fit!(test_model, data, Φ, max_iters=200)


DimensionMismatch: DimensionMismatch: A has dimensions (1,20000) but B has dimensions (3,1)

In [10]:
test_model.B[2].β

ErrorException: type GaussianEmission has no field β

In [11]:
println(test_model.A)   
println(true_model.A)
println(test_model.B[1].β)
println(test_model.B[2].β)

[0.8 0.2; 0.05 0.95]
[0.9 0.1; 0.2 0.8]


ErrorException: type GaussianEmission has no field β

In [12]:
plot(ll[3:end])

UndefVarError: UndefVarError: `ll` not defined

# Switching Poisson Regression

In [13]:
"""
Create a Switching Poisson Regression
"""
# Create the emission models
emission_1 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([4, 3, 2, 4], :, 1))
emission_2 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([-4, -2, 1, 3], :, 1))

# Initialize the SwitchingPoissonRegression
true_model = SwitchingPoissonRegression(K=2, input_dim=3, output_dim=1)

# Plug in the emission models
true_model.B[1] = emission_1
true_model.B[2] = emission_2

# Sample from the HMM
n=20000
Φ = randn(n, 3)
true_labels, data = SSD.sample(true_model, Φ, n=n)

# Create a new SwitchingPoissonRegression and try to recover parameters
test_model = SwitchingPoissonRegression(K=2, input_dim=3, output_dim=1)

# Create the emission models for warm start
emission_1 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([2.0, 1.0, 4.0, 2.0], :, 1))
emission_2 = PoissonRegressionEmission(input_dim=3, output_dim=1, include_intercept=true, β=reshape([-5.0, -1.0, 0.0, 2.0], :, 1))
test_model.B[1], test_model.B[2] = emission_1, emission_2

ll = SSD.fit!(test_model, data, Φ, max_iters=200)

print(isapprox(test_model.B[1].β, true_model.B[1].β, atol=0.1) || isapprox(test_model.B[1].β, true_model.B[2].β, atol=0.1))
print(isapprox(test_model.B[2].β, true_model.B[2].β, atol=0.1) || isapprox(test_model.B[2].β, true_model.B[1].β, atol=0.1))


DimensionMismatch: DimensionMismatch: matrix A has dimensions (1,20000), matrix B has dimensions (4,1)

In [14]:
plot(ll)

UndefVarError: UndefVarError: `plot` not defined

In [15]:
# Create Guassian Emission Models
output_dim = 2
μ = [0.0, 0.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(output_dim, μ, Σ)

μ = [2.0, 1.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(output_dim, μ, Σ)

# Create GaussianHMM
true_model = GaussianHMM(K=2, output_dim=2)
true_model.B[1] = emission_1
true_model.B[2] = emission_2
true_model.A = [0.9 0.1; 0.8 0.2]

# Sample from the model
n=20000
true_labels, data = StateSpaceDynamics.sample(true_model, n=n)

# Fit a new gaussian hmm to the data
test_model = GaussianHMM(K=2, output_dim=2)
ll = SSD.fit!(test_model, data)

CompositeException: TaskFailedException

    nested task error: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 20000 and 2
    Stacktrace:
      [1] _bcs1
        @ .\broadcast.jl:555 [inlined]
      [2] _bcs
        @ .\broadcast.jl:549 [inlined]
      [3] broadcast_shape
        @ .\broadcast.jl:543 [inlined]
      [4] combine_axes
        @ .\broadcast.jl:524 [inlined]
      [5] instantiate
        @ .\broadcast.jl:306 [inlined]
      [6] materialize
        @ .\broadcast.jl:903 [inlined]
      [7] fit!(model::GaussianEmission, Y::Matrix{Float64}, w::Vector{Float64})
        @ StateSpaceDynamics C:\Users\ryansenne\Documents\GitHub\ssm_julia\src\EmissionModels.jl:125
      [8] macro expansion
        @ C:\Users\ryansenne\Documents\GitHub\ssm_julia\src\HiddenMarkovModels.jl:291 [inlined]
      [9] (::StateSpaceDynamics.var"#321#threadsfor_fun#100"{StateSpaceDynamics.var"#321#threadsfor_fun#99#101"{HiddenMarkovModel, Matrix{Float64}, Tup

In [16]:
println(true_model.A)
println(test_model.A)

[0.9 0.1; 0.8 0.2]
[0.12071916710001702 0.8792808329012788; 0.9753464170187637 0.024653582981984033]
